# Automated Project: Planning, Estimation, and Allocation

## Initial Imports

In [ ]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

# Load environment variables
from helper import load_env
load_env()

import os
import yaml
from crewai import Agent, Task, Crew

## Set OpenAI Model

In [ ]:
os.environ['OPENAI_MODEL_NAME'] = 'gpt-4o-mini'

## Loading Tasks and Agents YAML files

In [ ]:
# Define file paths for YAML configurations
files = {
    'agents': 'config/agents.yaml',
    'tasks': 'config/tasks.yaml'
}

# Load configurations from YAML files
configs = {}
for config_type, file_path in files.items():
    with open(file_path, 'r') as file:
        configs[config_type] = yaml.safe_load(file)

# Assign loaded configurations to specific variables
agents_config = configs['agents']
tasks_config = configs['tasks']

## Create Pydantic Models for Structured Output

In [ ]:
from typing import List
from pydantic import BaseModel, Field

class TaskEstimate(BaseModel):
    task_name: str = Field(..., description="Name of the task")
    estimated_time_hours: float = Field(..., description="Estimated time to complete the task in hours")
    required_resources: List[str] = Field(..., description="List of resources required to complete the task")

class Milestone(BaseModel):
    milestone_name: str = Field(..., description="Name of the milestone")
    tasks: List[str] = Field(..., description="List of task IDs associated with this milestone")

class ProjectPlan(BaseModel):
    tasks: List[TaskEstimate] = Field(..., description="List of tasks with their estimates")
    milestones: List[Milestone] = Field(..., description="List of project milestones")

## Create Crew, Agents and Tasks

In [ ]:
# Creating Agents
project_planning_agent = Agent(
  config=agents_config['project_planning_agent']
)

estimation_agent = Agent(
  config=agents_config['estimation_agent']
)

resource_allocation_agent = Agent(
  config=agents_config['resource_allocation_agent']
)

# Creating Tasks
task_breakdown = Task(
  config=tasks_config['task_breakdown'],
  agent=project_planning_agent
)

time_resource_estimation = Task(
  config=tasks_config['time_resource_estimation'],
  agent=estimation_agent
)

resource_allocation = Task(
  config=tasks_config['resource_allocation'],
  agent=resource_allocation_agent,
  output_pydantic=ProjectPlan # This is the structured output we want
)

# Creating Crew
crew = Crew(
  agents=[
    project_planning_agent,
    estimation_agent,
    resource_allocation_agent
  ],
  tasks=[
    task_breakdown,
    time_resource_estimation,
    resource_allocation
  ],
  verbose=True
)

## Crew's Inputs

In [ ]:
from IPython.display import display, Markdown

project = 'Website'
industry = 'Technology'
project_objectives = 'Create a website for a small business'
team_members = """
- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)
"""
project_requirements = """
- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust
"""

# Format the dictionary as Markdown for a better display in Jupyter Lab
formatted_output = f"""
**Project Type:** {project}

**Project Objectives:** {project_objectives}

**Industry:** {industry}

**Team Members:**
{team_members}
**Project Requirements:**
{project_requirements}
"""
# Display the formatted output as Markdown
display(Markdown(formatted_output))


**Project Type:** Website

**Project Objectives:** Create a website for a small business

**Industry:** Technology

**Team Members:**

- John Doe (Project Manager)
- Jane Doe (Software Engineer)
- Bob Smith (Designer)
- Alice Johnson (QA Engineer)
- Tom Brown (QA Engineer)

**Project Requirements:**

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build trust



## Kicking off the crew

In [ ]:
# The given Python dictionary
inputs = {
  'project_type': project,
  'project_objectives': project_objectives,
  'industry': industry,
  'team_members': team_members,
  'project_requirements': project_requirements
}

# Run the crew
result = crew.kickoff(
  inputs=inputs
)

# Agent: The Ultimate Project Planner
## Task: Carefully analyze the project_requirements for the Website project and break them down into individual tasks. Define each task's scope in detail, set achievable timelines, and ensure that all dependencies are accounted for:

- Create a responsive design that works well on desktop and mobile devices
- Implement a modern, visually appealing user interface with a clean look
- Develop a user-friendly navigation system with intuitive menu structure
- Include an "About Us" page highlighting the company's history and values
- Design a "Services" page showcasing the business's offerings with descriptions
- Create a "Contact Us" page with a form and integrated map for communication
- Implement a blog section for sharing industry news and company updates
- Ensure fast loading times and optimize for search engines (SEO)
- Integrate social media links and sharing capabilities
- Include a testimonials section to showcase customer feedback and build tru



# Agent: Resource Allocation Strategist
## Final Answer: 
### Resource Allocation Chart for Website Project

| **Task**                      | **Assigned To**              | **Start Date** | **End Date**   |
|-------------------------------|-------------------------------|-----------------|-----------------|
| Kick-off Meeting              | John Doe                     | 01/01/2024      | 01/01/2024      |
| Identify Project Requirements   | John Doe                     | 01/02/2024      | 01/03/2024      |
| Create Responsive Design      | Bob Smith                    | 01/04/2024      | 01/08/2024      |
| Implement User Interface      | Jane Doe                     | 01/09/2024      | 01/12/2024      |
| Develop Navigation System     | Jane Doe                     | 01/10/2024      | 01/11/2024      |
| Create "About Us" Page       | Bob Smith                    | 01/12/2024      | 01/12/2024      |
| Design "Services" Page       | Bob Smith                    | 01/13/2024      |

## Usage Metrics and Costs

Let’s see how much it would cost each time if this crew runs at scale.

In [ ]:
import pandas as pd

costs = 0.150 * (crew.usage_metrics.prompt_tokens + crew.usage_metrics.completion_tokens) / 1_000_000
print(f"Total costs: ${costs:.4f}")

# Convert UsageMetrics instance to a DataFrame
df_usage_metrics = pd.DataFrame([crew.usage_metrics.dict()])
df_usage_metrics

Total costs: $0.0033


,total_tokens,prompt_tokens,completion_tokens,successful_requests
0,22128,13340,8788,9


## Result

In [ ]:
result.pydantic.dict()

{'tasks': [{'task_name': 'Kick-off Meeting',
   'estimated_time_hours': 1.0,
   'required_resources': ['John Doe']},
  {'task_name': 'Identify Project Requirements',
   'estimated_time_hours': 24.0,
   'required_resources': ['John Doe', 'Stakeholders']},
  {'task_name': 'Create Responsive Design',
   'estimated_time_hours': 32.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Implement User Interface',
   'estimated_time_hours': 48.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Develop Navigation System',
   'estimated_time_hours': 32.0,
   'required_resources': ['Jane Doe']},
  {'task_name': 'Create "About Us" Page',
   'estimated_time_hours': 8.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Design "Services" Page',
   'estimated_time_hours': 8.0,
   'required_resources': ['Bob Smith']},
  {'task_name': 'Create "Contact Us" Page',
   'estimated_time_hours': 8.0,
   'required_resources': ['Jane Doe', 'Content Writer']},
  {'task_name': 'Develop Blo

## Inspect further

In [ ]:
tasks = result.pydantic.dict()['tasks']
df_tasks = pd.DataFrame(tasks)

# Display the DataFrame as an HTML table
df_tasks.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,task_name,estimated_time_hours,required_resources
0,Kick-off Meeting,1.000000,['John Doe']
1,Identify Project Requirements,24.000000,"['John Doe', 'Stakeholders']"
2,Create Responsive Design,32.000000,['Bob Smith']
3,Implement User Interface,48.000000,['Jane Doe']
4,Develop Navigation System,32.000000,['Jane Doe']
5,"Create ""About Us"" Page",8.000000,['Bob Smith']
6,"Design ""Services"" Page",8.000000,['Bob Smith']
7,"Create ""Contact Us"" Page",8.000000,"['Jane Doe', 'Content Writer']"
8,Develop Blog Section,32.000000,['Jane Doe']
9,Speed and SEO Optimization,24.000000,['Jane Doe']


### Inspecting Milestones

In [ ]:
milestones = result.pydantic.dict()['milestones']
df_milestones = pd.DataFrame(milestones)

# Display the DataFrame as an HTML table
df_milestones.style.set_table_attributes('border="1"').set_caption("Task Details").set_table_styles(
    [{'selector': 'th, td', 'props': [('font-size', '120%')]}]
)

,milestone_name,tasks
0,Project Kick-off,['Kick-off Meeting']
1,Requirements Finalized,['Identify Project Requirements']
2,Design Phase Complete,"['Create Responsive Design', 'Create ""About Us"" Page', 'Design ""Services"" Page', 'Design Testimonials Section']"
3,Development Phase Complete,"['Implement User Interface', 'Develop Navigation System', 'Create ""Contact Us"" Page', 'Develop Blog Section', 'Speed and SEO Optimization', 'Integrate Social Media Links']"
4,Testing and QA Complete,['Testing and QA']
5,Final Review Complete,['Final Review']
6,Website Launched,['Launch Website']
